<a href="https://colab.research.google.com/github/khusheedhanuka/recommendation_systems/blob/main/Hybrid_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np


num_users = 10
num_movies = 10

ratings = np.random.randint(1, 6, size=(num_users, num_movies))


user_ids = ['User_' + str(i) for i in range(1, num_users + 1)]
movie_ids = ['Movie_' + str(i) for i in range(1, num_movies + 1)]

df = pd.DataFrame(ratings, index=user_ids, columns=movie_ids)



In [ ]:
df

,Movie_1,Movie_2,Movie_3,Movie_4,Movie_5,Movie_6,Movie_7,Movie_8,Movie_9,Movie_10
User_1,5,5,4,4,2,1,5,3,5,2
User_2,3,5,2,1,2,2,3,2,4,3
User_3,3,5,3,4,5,2,1,5,1,4
User_4,4,4,4,4,2,1,1,3,3,5
User_5,3,4,1,1,1,5,3,4,3,4
User_6,1,5,3,5,1,3,1,1,5,3
User_7,3,5,5,2,5,3,4,1,2,2
User_8,5,5,3,3,1,2,2,3,2,1
User_9,2,5,1,2,3,3,2,1,3,5
User_10,3,1,2,3,1,3,4,4,3,1


In [ ]:
num_features= 5

movie_features = np.random.randint(1, 6, size=(num_movies, num_features))

movie_features_df = pd.DataFrame(movie_features, index=movie_ids, columns=[f'Feature_{i}' for i in range(1, num_features + 1)])

movie_features_df

,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5
Movie_1,3,3,1,5,5
Movie_2,2,3,2,1,2
Movie_3,3,3,1,2,3
Movie_4,1,2,3,3,4
Movie_5,1,2,4,3,2
Movie_6,5,5,5,4,5
Movie_7,4,1,3,4,3
Movie_8,4,1,2,5,3
Movie_9,2,4,5,1,5
Movie_10,2,5,1,1,5


In [ ]:
# import numpy as np
# import tensorflow as tf
# from tensorflow.keras.layers import Input, Dense
# from tensorflow.keras.models import Model
# from sklearn.preprocessing import MinMaxScaler

# # Example user-item matrix
# user_item_matrix = np.array([
#     [5, 3, 0, 1],
#     [4, 0, 0, 1],
#     [1, 1, 0, 5],
#     [1, 0, 0, 4],
#     [0, 1, 5, 4],
# ])

# user_item_matrix_scaled= MinMaxScaler().fit_transform(user_item_matrix)


# # Define the size of the input and the encoding dimension
# input_dim = user_item_matrix.shape[1]
# encoding_dim = 2  # This can be tuned

# # Define the autoencoder model
# input_layer = Input(shape=(input_dim,))
# encoder = Dense(encoding_dim, activation='relu')(input_layer)
# decoder = Dense(input_dim, activation='sigmoid')(encoder)

# autoencoder = Model(inputs=input_layer, outputs=decoder)

# # Compile the model
# autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# # Train the model
# autoencoder.fit(user_item_matrix_scaled, user_item_matrix_scaled,
#                 epochs=50,
#                 batch_size=2,
#                 shuffle=True)

# # Get the encoded and decoded data
# encoded_data = autoencoder.predict(user_item_matrix)
# decoded_data = autoencoder.predict(encoded_data)

# print("Original User-Item Matrix:")
# print(user_item_matrix)
# print("Reconstructed User-Item Matrix:")
# print(decoded_data*10)

# **Collaborative** **Rec**

In [ ]:
def matrix_factorization(R, P, Q, K, steps=100, alpha=0.002, beta=0.001):
    Q = Q.T
    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):

                if R[i][j] > 0:
                    eij = R[i][j] - np.dot(P[i, :], Q[:, j])
                    for k in range(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eR = np.dot(P, Q)
        e = 0
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - np.dot(P[i, :], Q[:, j]), 2)
                    for k in range(K):
                        e = e + (beta / 2) * (pow(P[i][k], 2) + pow(Q[k][j], 2))
        if e < 0.001:
            break
    return P, Q.T

In [ ]:
df_train=df.to_numpy()

N=len(df_train)

M=len(df_train[0])

K=8

P=np.random.rand(N,K)

Q=np.random.rand(M,K)

nP,nQ=matrix_factorization(df_train,P,Q,K)

In [ ]:
train_pred=np.dot(nP,nQ.T)
print(train_pred.round())

[[5. 5. 4. 4. 3. 2. 5. 3. 4. 2.]
 [3. 5. 2. 2. 3. 3. 3. 2. 3. 3.]
 [3. 5. 4. 4. 4. 2. 1. 4. 2. 4.]
 [4. 4. 4. 4. 2. 1. 1. 3. 3. 4.]
 [3. 4. 1. 1. 1. 5. 3. 4. 3. 4.]
 [2. 5. 3. 4. 1. 3. 1. 2. 4. 3.]
 [3. 5. 4. 2. 4. 2. 4. 2. 3. 3.]
 [4. 4. 3. 3. 1. 2. 2. 3. 3. 2.]
 [2. 5. 2. 2. 3. 3. 2. 2. 3. 4.]
 [3. 2. 2. 3. 1. 3. 4. 3. 3. 1.]]


In [ ]:
df_train

array([[5, 5, 4, 4, 2, 1, 5, 3, 5, 2],
       [3, 5, 2, 1, 2, 2, 3, 2, 4, 3],
       [3, 5, 3, 4, 5, 2, 1, 5, 1, 4],
       [4, 4, 4, 4, 2, 1, 1, 3, 3, 5],
       [3, 4, 1, 1, 1, 5, 3, 4, 3, 4],
       [1, 5, 3, 5, 1, 3, 1, 1, 5, 3],
       [3, 5, 5, 2, 5, 3, 4, 1, 2, 2],
       [5, 5, 3, 3, 1, 2, 2, 3, 2, 1],
       [2, 5, 1, 2, 3, 3, 2, 1, 3, 5],
       [3, 1, 2, 3, 1, 3, 4, 4, 3, 1]])

In [ ]:
pred=train_pred
testset=df_train
total=0
jum=0
for i in range(len(testset)):
    for j in range(len(testset[i])):
        if testset[i][j]>0:
            total+=pow(testset[i][j]-pred[i][j],2)
            jum+=1
mse=total/jum
print(mse)
rmse=np.sqrt(mse)
print(rmse)


0.26023900537350586
0.5101362615747932


In [ ]:
def collaborative_filtering_recommendations(user_id, user_item_matrix, top_n=10):

    user_preferences = user_item_matrix[user_id]


    sorted_items = sorted(range(len(user_preferences)), key=lambda i: user_preferences[i], reverse=True)


    top_recommendations = sorted_items[:top_n]

    return top_recommendations

In [ ]:
collaborative_filtering_recommendations(0,train_pred,4)

[1, 0, 6, 2]

# **Content** **Based** **Recommendation**

In [ ]:
#import linear kerenel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel,cosine_similarity
from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# normalized_features = scaler.fit_transform(movie_features_df)
# movie_features_df = pd.DataFrame(normalized_features, columns=movie_features_df.columns)

cosine_sim=cosine_similarity(movie_features_df)

In [ ]:
cosine_sim.round(2)

array([[1.  , 0.82, 0.94, 0.91, 0.78, 0.89, 0.89, 0.93, 0.76, 0.84],
       [0.82, 1.  , 0.94, 0.85, 0.84, 0.97, 0.81, 0.75, 0.94, 0.91],
       [0.94, 0.94, 1.  , 0.85, 0.76, 0.95, 0.87, 0.86, 0.84, 0.92],
       [0.91, 0.85, 0.85, 1.  , 0.93, 0.92, 0.87, 0.84, 0.91, 0.81],
       [0.78, 0.84, 0.76, 0.93, 1.  , 0.91, 0.86, 0.81, 0.88, 0.66],
       [0.89, 0.97, 0.95, 0.92, 0.91, 1.  , 0.92, 0.88, 0.93, 0.86],
       [0.89, 0.81, 0.87, 0.87, 0.86, 0.92, 1.  , 0.98, 0.76, 0.65],
       [0.93, 0.75, 0.86, 0.84, 0.81, 0.88, 0.98, 1.  , 0.67, 0.63],
       [0.76, 0.94, 0.84, 0.91, 0.88, 0.93, 0.76, 0.67, 1.  , 0.87],
       [0.84, 0.91, 0.92, 0.81, 0.66, 0.86, 0.65, 0.63, 0.87, 1.  ]])

In [ ]:
def get_recommendaions(index,cosine_sim=cosine_sim):

  title='Movie_'+str(index)
  sim_scores=list(enumerate(cosine_sim[index]))
  sim_scores=sorted(sim_scores,key=lambda x:x[1],reverse=True)
  sim_scores=sim_scores[1:5]
  sim_index=[i[0] for i in sim_scores]
  print(title)
  return sim_index

In [ ]:
sim=get_recommendaions(1)
sim

Movie_1


[5, 2, 8, 9]

## **Hybrid Recommendation **

In [ ]:
alpha=0.3
beta=0.7

In [ ]:
collab=collaborative_filtering_recommendations(0,train_pred,4)
content=get_recommendaions(0)

Movie_0


In [ ]:
def weighted_hybrid_recommendations(collaborative_recommendations, content_based_recommendations, weight_collaborative, weight_content_based, top_n=10):

    combined_scores = {}


    for movie_id in collaborative_recommendations:
        combined_scores[movie_id] = weight_collaborative


    for movie_id in content_based_recommendations:

        if movie_id in combined_scores:
            combined_scores[movie_id] += weight_content_based
        else:
            combined_scores[movie_id] = weight_content_based


    sorted_movies = sorted(combined_scores.keys(), key=lambda x: combined_scores[x], reverse=True)


    return sorted_movies[:top_n]

In [ ]:
recommended_movies = weighted_hybrid_recommendations(collab, content, alpha, beta, top_n=5)

recommended_movies

[2, 7, 3, 5, 1]